In [ ]:
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")

import numpy as np
from visu import Visualizer
import imageio


# STD
import os
import copy

# MISC
import numpy as np
import torch
import imageio
import cv2
from PIL import Image

# matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib.patches as patches

In [ ]:
from PIL import Image
import cv2
import torch
from torchvision import transforms as tf

def readFlowKITTI(filename):
    flow = cv2.imread(filename, cv2.IMREAD_ANYDEPTH|cv2.IMREAD_COLOR)
    flow = flow[:,:,::-1].astype(np.float32)
    flow, valid = flow[:, :, :2], flow[:, :, 2].astype(bool)
    flow = (flow - 2**15) / 64.0
    H,W = 960,1280
    return flow, valid
#flow, valid = readFlowKITTI(flow_pths[10])

# GENERATION METHOD FLOW
def load_image(imfile):
    img = np.array(Image.open(imfile)).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    img = img[:, 12:-12, 12:-12]
    return img[None]

# GENERATION METHOD LABELS
H,W = 640, 1280
def readImage(filename, H=640, W=1280, scale=True):
    _crop_center = tf.CenterCrop((H,W))
    img = Image.open(filename)    
    if scale:
        img = _crop_center( img )
    return np.array( img )   

from utils_asl import LabelLoaderAuto
lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet")



In [ ]:
visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=False, num_classes=41)

i0= "/home/jonfrey/Datasets/scannet/scans/scene0000_00/color/300.jpg"
l0 = "/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained/scans/scene0000_00/create_labels_from_pretrained/300.png"
f0 = "/home/jonfrey/Datasets/scannet/scans/scene0000_00/flow_sub_10/flow_up_300.png"
l0_gt = "/home/jonfrey/Datasets/scannet/scans/scene0000_00/label-filt/300.png"
def get_aligned_flow(f):
    return readFlowKITTI(f)
def get_alinged_label(l):
    return lla.get(l)[0][12:-12, 12:-12]

def get_aligned_image(i):
    return readImage(i, H=640, W=1280, scale=False)[ 12:-12, 12:-12,:]
def get_alinged_label_probs(l):
    return lla.get_probs(l)[12:-12, 12:-12]

f,l,i,l_gt = get_aligned_flow(f0),get_alinged_label(l0),get_aligned_image(i0),get_alinged_label(l0_gt)
l_soft = get_alinged_label_probs(l0)

textwidth = 12.7 #cm
fig, ax = plt.subplots(1, 4, figsize=(textwidth, 2.3), sharex=False, sharey=False)
left  = 0.05  # the left side of the subplots of the figure
right = 0.99    # the right side of the subplots of the figure
bottom = 0.3   # the bottom of the subplots of the figure
top = 0.88      # the top of the subplots of the figure
wspace = 0.25   # the amount of width reserved for blank space between subplots
hspace = 0.2  # the amount of height reserved for white space between subplots
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)


ax[0].imshow( visu.plot_image(i,jupyter=False))
ax[1].imshow(  visu.plot_flow(f[0],jupyter=False))
ax[2].imshow( visu.plot_detectron(i, l,jupyter=False))
ax[3].imshow( visu.plot_detectron(i, l_gt,jupyter=False))

for a in fig.axes:
    a.axis('off')

In [ ]:
from skimage.segmentation import slic
l = slic(i,n_segments=40, compactness=10.0)
res = visu.plot_detectron(i, l,jupyter=True,text_off =True)

In [ ]:
from pathlib import Path

i0= "/home/jonfrey/Datasets/scannet/scans/scene0000_00/color/0.jpg"
l0 = "/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained/scans/scene0000_00/create_labels_from_pretrained/0.png"
f0 = "/home/jonfrey/Datasets/scannet/scans/scene0000_00/flow_sub_10/flow_up_0.png"
l0_gt = "/home/jonfrey/Datasets/scannet/scans/scene0000_00/label-filt/0.png"

images = [str(s) for s in Path( "/home/jonfrey/Datasets/scannet/scans/scene0000_00/color/").rglob('*.jpg')]
images.sort(
    key=lambda x: int(x.split("/")[-3][-7:]) * 10000 + int(x.split("/")[-1][:-4])
)
flows = [ i.replace("color/","flow_sub_10/flow_up_").replace(".jpg",".png") for i in images]
labels_gt = [ f.replace("flow_sub_10/flow_up_","label-filt/") for f in flows]
labels_pred = [ f.replace("/home/jonfrey/Datasets/scannet/scans/scene0000_00/flow_sub_10/flow_up_","/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained/scans/scene0000_00/create_labels_from_pretrained/") for f in flows]

n = 0
f = get_aligned_flow(flows[n])
l_pred_hard = get_alinged_label(labels_pred[n])
i = get_aligned_image(images[n])
l_gt = get_alinged_label(labels_gt[n])
l_pred_soft = get_alinged_label_probs( labels_pred[n])

In [ ]:
import copy
class AccMonitor():
    def __init__(self):
        self.metrics = {}
        self.metric_dict = {'correct': 0,
                            'correct_valid': 0,
                            'correct_valid_both': 0,
                            'total': 0, 
                            'total_valid':0,
                            'total_valid_both':0,
                            'images': 0}
    def register(self,name):
        self.metrics[name] = copy.deepcopy(self.metric_dict)
        
    def update(self,name, label, gt):
        if name not in self.metrics.keys():
            self.register(name)
        m1 = gt != -1
        self.metrics[name]["total_valid"] += (m1).sum()
        self.metrics[name]["correct_valid"] += np.sum(gt[m1] == label[m1])
        
        m2 = (gt != -1 )*(label != -1)
        self.metrics[name]["total_valid"] += (m2).sum()
        self.metrics[name]["correct_valid"] += np.sum(gt[m2] == label[m2])
        
        self.metrics[name]["total"] += gt.size
        self.metrics[name]["correct"] += np.sum(gt == label)
        self.metrics[name]["images"] += 1
        
    def __str__(self):
        s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc} \n"  
        return s
        
accm = AccMonitor()
accm.update("Test", np.ones( (100,100)) ,np.ones( (100,100)))
accm.update("Test2", np.ones( (100,100)) ,np.ones( (100,100)))
print(accm)

In [ ]:
from torchvision.utils import make_grid
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

import copy
import torch
import cv2
import numpy as np
import imageio
import os

import matplotlib.pyplot as plt
"""
TODO:
- Depth
- BoundingBox Predicitons
"""
from torch.nn.functional import one_hot


def get_confidence_values( seq_length ):
    return [float( 1/seq_length)] * seq_length 

def calculate_label( index=None, seg=[],flow=[], img=[] ):
    seg_forwarded, img_forwarded = forward_labels( seg, flow, img)
    # -1 39 -> 0 -> 40 We assume that the network is also able to predict the invalid class
    # In reality this is not the case but this way we can use the ground truth labels for testing

    if seg_forwarded[0].shape[2] == 1:
        for i in range(len( seg_forwarded) ):
            seg_forwarded[i] += 1 

    confidence_values_list = get_confidence_values(seq_length= len(seg_forwarded))
    
    if seg_forwarded[0].shape[2] == 1:
        one_hot_acc = np.zeros( (*seg_forwarded[0].shape,40+1), dtype=np.float32) # H,W,C
        for conf, seg in zip(confidence_values_list, seg_forwarded):    
            one_hot_acc += (np.eye(40+1)[seg.astype(np.int32)]).astype(np.float32) * conf
        invalid_labels = np.sum( one_hot_acc[:,:,0,1:],axis=2 ) == 0
    else:
        res = np.argmax( np.stack( out ).sum( axis = 0 ), axis=2 )
        return res, seg_forwarded
    
    label = np.argmax( one_hot_acc[:,:,0,1:], axis=2 )
    label[ invalid_labels ] = -1 
    
    
    return label[:,:], [s[:,:,0] for s in seg_forwarded]


def forward_labels( seg=[],flow=[], img= [] ):
    """
    seg[0] , C,H,W   0 == lastes frame, -1 == oldest frame

    pre_fusion_function should be used to integrate the depth measurments 
    to the semseg before forward projection !

    seg_forwarded[0] -> oldest_frame
    seg_forwarded[len(seg_forwarded)] -> latest_frame not forwarded

    """
    C,_H,_W = seg[0].shape
    
    if len( seg[0].shape ) == 3 and seg[0].shape[0] != 1:
        soft = True
    else:
        soft = False
    
    seg_forwarded = []
    img_forwarded = []
    
    for j in range(len( seg )):
        seg[j] = np.moveaxis( seg[j], [0,1,2], [2,0,1] ) #C,H,W -> H,W,C
        # img[j] = np.moveaxis( img[j], [0,1,2], [2,0,1] ) #C,H,W -> H,W,C

    for i in range(0,len(seg)-1):
        i = len(seg)-1-i
        
        seg_forwarded.append( seg[i].astype(np.float32) )
        img_forwarded.append( img[i].astype(np.float32) )
        
        # start at oldest frame
        if i != 0:
            f = flow[i]
        else:
            f = np.zeros(flow[i].shape, dtype=np.float32)
        
        h_, w_ = np.mgrid[0:_H, 0:_W].astype(np.float32)
        h_ -= f[:,:,1]
        w_ -= f[:,:,0]

        j = 0
        for s,_img in zip(seg_forwarded, img_forwarded) : #  seg_forwarded, depth_forwarded
            if soft:
                s = cv2.remap( s, w_, h_, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
            else:
                # in H,W,1
                s = cv2.remap( s, w_, h_, interpolation=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=-1)[:,:,None]
                # out H,W,1
            img_forwarded[j] = cv2.remap( _img, w_, h_, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
            seg_forwarded[j] = s

            j += 1

    img_forwarded.append( img[0] )
    seg_forwarded.append( seg[0].astype(np.float32) )
    return seg_forwarded, img_forwarded

In [ ]:
import time

accm = AccMonitor()
live_plot = True
seg_ls=[]
flow_ls=[]
img_ls= []
lengths = [6,4,6,12]

start = 0

plotted_images = []
st = time.time()
for length in lengths:
    for f, l_p, l_gt, i in zip(flows[start:],labels_pred[start:],labels_gt[start:],images[start:]):
        if int(i.split('/')[-1][:-4])%10==0:
            if int(i.split('/')[-1][:-4]) == 5570: 
                break
            if int(i.split('/')[-1][:-4]) % 100 == 0:
                print("N:", int( len(flows)/10 ),"/",int(int(i.split('/')[-1][:-4])/10)," Time:", time.time()-st)
            flow = get_aligned_flow( f )[0]
            l_pred_hard = get_alinged_label( l_p )
            img = get_aligned_image( i )
            l_gt = get_alinged_label(l_gt )
            l_pred_soft = get_alinged_label_probs( l_p )

            seg_ls.append( np.argmax( np.moveaxis( l_pred_soft , [0,1,2], [1,2,0] ), axis=0)[None] )
            flow_ls.append( flow )
            img_ls.append( img )

            if len(seg_ls) > length:
                seg_ls = seg_ls[-length:]
                flow_ls = flow_ls[-length:]
                img_ls = img_ls[-length:]
            if len(seg_ls) == length:
                label, seg_forwarded = calculate_label( seg= copy.deepcopy(seg_ls)[::-1],
                                                       flow= copy.deepcopy(flow_ls)[::-1], 
                                                       img = copy.deepcopy(img_ls)[::-1] )

                if live_plot:
                    for s,im,ori_seg in zip( seg_forwarded, img_ls, seg_ls ) :
                        plotted_images.append( visu.plot_detectron(img, s , jupyter=True, text_off=True))
                        plotted_images.append( visu.plot_detectron(im, ori_seg[0]+1 , jupyter=True, text_off=True))
                    plotted_images.append(visu.plot_detectron(img, label+1, jupyter=True, text_off=True) )
                    plotted_images.append(visu.plot_detectron(img, l_gt, jupyter=True, text_off=True) )
                    plotted_images.append(visu.plot_detectron(img, seg_ls[-1][0]+1, jupyter=True, text_off=True) )


                accm.update( f"PSEUDO_{length}", label , l_gt-1)   
                accm.update( f"GT_{length}", l_gt-1, l_gt-1)   
                accm.update( f"NETWORK_{length}", l_pred_hard-1, l_gt-1)
                
                if live_plot: break
    if live_plot: break
    print(f"ACCM {length}:", accm)
print("Final", accm)

In [ ]:
for k,i in enumerate( plotted_images):
    Image.fromarray(i).save(f"docs/flow/a_{k}.png")

In [ ]:
textwidth = 12.7 #cm
fig, ax = plt.subplots(2, 4, figsize=(textwidth, 4.7), sharex=False, sharey=False)
left  = 0  # the left side of the subplots of the figure
right = 1    # the right side of the subplots of the figure
bottom = 0   # the bottom of the subplots of the figure
top = 1     # the top of the subplots of the figure
wspace = 0.02   # the amount of width reserved for blank space between subplots
hspace = 0.05  # the amount of height reserved for white space between subplots
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)

for i in range(0,8):
    ax[i%2,int(i/2)].imshow( plotted_images[i] )
for a in fig.axes:
    a.axis('off')
    
from visu.visualizer import get_img_from_fig
res = get_img_from_fig(fig, dpi=180)
imageio.imwrite( 'docs/optical_flow_propagation_4.png', res)

In [ ]:
textwidth = 12.7 #cm
fig, ax = plt.subplots(2, 6, figsize=(textwidth, 4), sharex=False, sharey=False)
left  = 0  # the left side of the subplots of the figure
right = 1    # the right side of the subplots of the figure
bottom = 0   # the bottom of the subplots of the figure
top = 1     # the top of the subplots of the figure
wspace = 0.08   # the amount of width reserved for blank space between subplots
hspace = 0.03  # the amount of height reserved for white space between subplots
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)

for i in range(0,12):
    ax[i%2,int(i/2)].imshow( plotted_images[i] )
for a in fig.axes:
    a.axis('off')
    
from visu.visualizer import get_img_from_fig
res = get_img_from_fig(fig, dpi=360)
imageio.imwrite( 'docs/optical_flow_propagation_6.png', res)

In [ ]:
import copy
acc_noice = AccMonitor()
accm.update("Test2", np.ones( (100,100)) ,np.ones( (100,100)))


print(accm)


import time
visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=False, num_classes=41)
accm = AccMonitor()
live_plot = True
seg_ls=[]
flow_ls=[]
img_ls= []
gt_ls = []
start = 0

plotted_images = []
st = time.time()
length = 4
for f, l_p, l_gt, i in zip(flows[start:],labels_pred[start:],labels_gt[start:],images[start:]):
    if int(i.split('/')[-1][:-4])%10==0:
        if int(i.split('/')[-1][:-4]) == 5570: 
            break
        if int(i.split('/')[-1][:-4]) % 100 == 0:
            print("N:", int( len(flows)/10 ),"/",int(int(i.split('/')[-1][:-4])/10)," Time:", time.time()-st)
        flow = get_aligned_flow( f )[0]
        l_pred_hard = get_alinged_label( l_p )
        img = get_aligned_image( i )
        l_gt = get_alinged_label(l_gt )
        
        
        l_pred_soft = get_alinged_label_probs( l_p )
        
        seg_inp = l_gt[None]-1
        
        rand_label = np.random.randint(-1,39, seg_inp.shape )
        val = 0.5
        rand_mask = np.random.random( seg_inp.shape) > val
        seg_inp[rand_mask] = rand_label[rand_mask]
        
        gt_ls.append( l_gt)
        seg_ls.append( seg_inp)
        flow_ls.append( flow )
        img_ls.append( img )

        if len(seg_ls) > length:
            gt_ls= gt_ls[-length:]
            seg_ls = seg_ls[-length:]
            flow_ls = flow_ls[-length:]
            img_ls = img_ls[-length:]
        if len(seg_ls) == length:
            label, seg_forwarded = calculate_label( seg= copy.deepcopy(seg_ls)[::-1],
                                                   flow= copy.deepcopy(flow_ls)[::-1], 
                                                   img = copy.deepcopy(img_ls)[::-1] )

            if live_plot:
                for s,im,ori_seg,gg in zip( seg_forwarded, img_ls, seg_ls,l_gt ) :
                    print("Plot")
                    plotted_images.append( visu.plot_detectron(img, s , draw_bound=False, jupyter=False, text_off=True))
                    plotted_images.append( visu.plot_detectron(im, ori_seg[0]+1, draw_bound=False , jupyter=False, text_off=True))  
            accm.update( f"PSEUDO_{length}", label , l_gt-1)   
            accm.update( f"GT_{length}", seg_inp[0], l_gt-1)   
            accm.update( f"NETWORK_{length}", l_pred_hard-1, l_gt-1)
            plotted_images.append(visu.plot_detectron_true_false( img = img, pred = seg_ls[-1][0]+1, gt = l_gt, draw_bound=False, jupyter=False, text_off=True))
            print("Done")
            if live_plot: break

In [ ]:
wrong = []
for i in range( len(img_ls)):
    wrong.append( 
        visu.plot_detectron_true_false( img = img_ls[-1], pred =seg_forwarded[i] , gt = gt_ls[-1], draw_bound=False, jupyter=False, text_off=True)
    )

In [ ]:
wrong_static = []
for i in range( len(img_ls)):
    wrong_static.append( 
        visu.plot_detectron_true_false(img = img_ls[i], pred =seg_ls[i][0]+1 , gt = gt_ls[i], draw_bound=False, jupyter=True, text_off=True)
    )

In [ ]:
final = visu.plot_detectron(img, label+1, draw_bound=False, jupyter=True, text_off=True) 
fp_best = visu.plot_detectron_true_false( img = img, pred = label+1 , gt = l_gt, draw_bound=False, jupyter=True, text_off=True)


In [ ]:
for k,i in enumerate( plotted_images+wrong):
    Image.fromarray(i).save(f"docs/flow/b_{k}.png")

In [ ]:
textwidth = 12.7 #cm
fig, ax = plt.subplots(2, 5, figsize=(textwidth, 3.8), sharex=False, sharey=False)
left  = 0  # the left side of the subplots of the figure
right = 1    # the right side of the subplots of the figure
bottom = 0   # the bottom of the subplots of the figure
top = 1     # the top of the subplots of the figure
wspace = 0.04   # the amount of width reserved for blank space between subplots
hspace = 0.03  # the amount of height reserved for white space between subplots
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)

ax[0,0].imshow( plotted_images[0] )
ax[1,0].imshow( wrong[0] )
#ax[2,0].imshow( plotted_images[1] )
#ax[3,0].imshow( wrong_static[0] )

ax[0,1].imshow( plotted_images[2] )
ax[1,1].imshow( wrong[1] )
#ax[2,1].imshow( plotted_images[3] )
#ax[3,1].imshow( wrong_static[0] )

ax[0,2].imshow( plotted_images[4] )
ax[1,2].imshow( wrong[2] )
#ax[2,2].imshow( plotted_images[5] )
#ax[3,2].imshow( wrong_static[0] )

ax[0,3].imshow( plotted_images[6] )
ax[1,3].imshow( wrong[3] )
#ax[2,3].imshow( plotted_images[7] )
#ax[3,3].imshow( wrong_static[0] )

ax[0,4].imshow( final )
ax[1,4].imshow( fp_best )

for a in fig.axes:
    a.axis('off')
    
from visu.visualizer import get_img_from_fig
res = get_img_from_fig(fig, dpi=360)
imageio.imwrite( 'docs/random_noice_flow_res360.png', res)

res = get_img_from_fig(fig, dpi=180)
imageio.imwrite( 'docs/random_noice_flow_res180.png', res)
